# Customer Journey Analysis
# This notebook analyzes customer journeys across different products, visualizing patterns in purchasing behavior, demographics, and product adoption sequences.

## Import and plot-style

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pathlib import Path
import jax
import jax.numpy as jnp
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go

# Set style for better visualizations
plt.style.use("seaborn-v0_8-dark-palette")
sns.set_palette("husl")
%matplotlib inline


## Data Loading

In [ ]:
# We'll load all ABT_score files and combine them with appropriate target labels.
def load_abt_files():
    """Load all ABT_score files and combine them with appropriate target labels"""
    # List all ABT_Score/ABT_score files in current directory (case insensitive)
    abt_files = list(Path('../data').glob('ABT_[Ss]core_*.csv'))
    
    # Check if any files were found
    if not abt_files:
        print("No ABT_score_*.csv files found in current directory!")
        print("\nCurrent directory contents:")
        print([f.name for f in Path('../data').glob('*')])
        print("\nPlease ensure your ABT_score_*.csv files are in the data directory.")
        return None
    
    print(f"Found {len(abt_files)} ABT_score files:")
    for f in abt_files:
        print(f"  - {f.name}")
    
    dfs = []
    for file_path in abt_files:
        product = file_path.stem.split('_')[-1]
        try:
            print(f"\nLoading {product} data...")
            df = pd.read_csv(file_path, sep=';')
            print(f"Successfully loaded {len(df)} rows for {product}")
            df['product_type'] = product
            dfs.append(df)
        except Exception as e:
            print(f"Error loading {file_path.name}: {str(e)}")
    
    if not dfs:
        raise ValueError("No data frames were successfully loaded. Please check the file format and contents.")
    
    return pd.concat(dfs, ignore_index=True)

# Load the data
combined_df = load_abt_files()

if combined_df is not None:
    # Display basic information about the dataset
    print("\nDataset Overview:")
    print(f"Total number of records: {len(combined_df)}")
    print("\nProduct distribution:")
    print(combined_df['product_type'].value_counts())
else:
    print("\nPlease fix the data loading issues before continuing.")


## Data Preprocessing

In [ ]:
def preprocess_data(df):
    """Clean and preprocess the combined dataset without NaN flag columns"""
    df = df.copy()
    
    # Convert date columns to datetime (invalid -> NaT)
    date_columns = [col for col in df.columns if 'Date' in col or 'date' in col]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')  # Coerce invalid dates to NaT
    
    # Fill numeric NaNs with 0 (assuming 0 indicates absence)
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].fillna(0)
    
    # Convert binary columns to int (original NaN handling)
    binary_columns = [col for col in df.columns if col.startswith(('Have_', 'Had_', 'Optout_'))]
    for col in binary_columns:
        df[col] = df[col].fillna(0).astype(int)
    
    return df

def get_valid_data(df, column, include_converted_nans=False):
    """
    Get data excluding NaN values for specific column
    
    Parameters:
    df : DataFrame
    column : str - column name to analyze
    include_converted_nans : bool - if True, includes values that were converted from NaN to 0
    
    Returns:
    DataFrame with only valid data for the specified column
    """
    if f"{column}_was_nan" in df.columns and not include_converted_nans:
        return df[~df[f"{column}_was_nan"]]
    return df

def analyze_column_with_nans(df, column):
    """
    Analyze a column's statistics both including and excluding converted NaN values
    """
    print(f"\nAnalysis for column: {column}")
    print("----------------------------------------")
    
    # All data including converted NaNs
    print("Including converted NaN values:")
    print(df[column].describe())
    
    # Only original non-NaN values
    if f"{column}_was_nan" in df.columns:
        valid_data = get_valid_data(df, column)
        print("\nExcluding converted NaN values:")
        print(valid_data[column].describe())
        
        print(f"\nTotal values: {len(df)}")
        print(f"Original NaN values: {df[f'{column}_was_nan'].sum()}")
        print(f"Valid values: {len(valid_data)}")
    else:
        print("\nNo NaN tracking available for this column")

combined_df = preprocess_data(combined_df)

# Display sample of preprocessed data
print("Sample of preprocessed data:")
display(combined_df.head())

## Customer Journey Analysis

In [ ]:
# Analyze the sequence of products purchased by customers.
def analyze_product_sequence(df):
    """Analyze product sequence using valid dates (NaT excluded)"""
    product_cols = [col for col in df.columns if col.startswith('mFirst_')]
    
    timeline_data = []
    for customer_id in df['sCustomerNaturalKey'].unique():
        customer_data = df[df['sCustomerNaturalKey'] == customer_id]
        
        products = []
        for col in product_cols:
            product = col.replace('mFirst_', '')
            date = customer_data[col].iloc[0]
            if pd.notna(date):  # Automatically excludes NaT
                products.append({
                    'sCustomerNaturalKey': customer_id,
                    'product': product,
                    'acquisition_date': date
                })
        
        products = sorted(products, key=lambda x: x['acquisition_date'])
        timeline_data.extend(products)
    
    return pd.DataFrame(timeline_data)

# Analyze product sequences
product_timeline = analyze_product_sequence(combined_df)

# Display summary of product sequences
print("Most common first products:")
display(product_timeline.groupby('sCustomerNaturalKey')
        .first()['product']
        .value_counts()
        .head(10))


## Visualizations

In [ ]:
# Customer Journey Sankey Diagram
def plot_customer_journey_sankey(df):
    """Create a Sankey diagram of customer journeys"""
    product_sequence = analyze_product_sequence(df)
    
    # Group by customer and create product sequences
    customer_sequences = product_sequence.groupby('sCustomerNaturalKey').agg(
        list
    )['product'].value_counts().head(10)  # Top 10 most common sequences
    
    fig = go.Figure(data=[go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color = "black", width = 0.5),
            label = customer_sequences.index,
            color = "blue"
        ),
        link = dict(
            source = [i for i in range(len(customer_sequences)-1)],
            target = [i+1 for i in range(len(customer_sequences)-1)],
            value = customer_sequences.values[:-1]
        )
    )])
    
    fig.update_layout(title_text="Most Common Customer Journey Paths", 
                     font_size=10,
                     height=600)
    fig.show()

# Create Sankey diagram
plot_customer_journey_sankey(combined_df)

In [ ]:
# Demographic Analysis
def plot_demographic_distribution(df):
    """Plot age distribution for different products (exclude 0 values)"""
    # Filter out 0 ages (original NaNs)
    df_valid_age = df[df['Age'] > 0]
    
    fig, ax1 = plt.subplots(figsize=(10, 6))
    sns.boxplot(x='product_type', y='Age', data=df_valid_age, ax=ax1)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)
    ax1.set_title('Age Distribution by Product (Excluding Missing Values)')
    plt.show()
    
    print("\nMean age by product (excluding missing values):")
    display(df_valid_age.groupby('product_type')['Age'].mean().sort_values(ascending=False))
    
plot_demographic_distribution(combined_df)

In [ ]:
# Product Adoption Timeline
def plot_product_adoption_timeline(df):
    """Plot timeline using valid dates only"""
    timeline_data = analyze_product_sequence(df)
    
    # Filter out any remaining invalid dates (shouldn't be any)
    timeline_data = timeline_data[pd.notna(timeline_data['acquisition_date'])]
    
    fig = px.scatter(timeline_data, 
                    x='acquisition_date', 
                    y='product',
                    color='product',
                    title='Product Adoption Timeline')
    fig.update_layout(height=600)
    fig.show()
    
plot_product_adoption_timeline(combined_df)

## Optional: Predictive Modeling

In [ ]:
# We can use PyTorch to build a model predicting future product adoption.
class CustomerJourneyPredictor(torch.nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(input_size, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(64, 1),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers(x)

def prepare_features(df):
    """Prepare features for the prediction model"""
    feature_cols = [col for col in df.columns if col.startswith(('Have_', 'Had_', 'nbr_active_agr_'))]
    X = df[feature_cols]
    y = df['myTarget']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return torch.FloatTensor(X_scaled), torch.FloatTensor(y.values)

In [ ]:
# Prepare data and initialize model
X, y = prepare_features(combined_df)
model = CustomerJourneyPredictor(X.shape[1])
print("Model architecture:")
print(model)

## Additional Insights

In [ ]:
# Product combinations analysis
def analyze_product_combinations(df):
    """Analyze which products are commonly held together"""
    have_cols = [col for col in df.columns if col.startswith('Have_')]
    product_combinations = df[have_cols].sum()
    
    # Create correlation matrix
    corr_matrix = df[have_cols].corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Product Combination Correlations')
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    return product_combinations

print("Product ownership analysis:")
display(analyze_product_combinations(combined_df))